# VGG5 SpinalNet (for MNIST)

In [1]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

class VGG(nn.Module):  
         
    def __init__(self, num_classes=10):
        super().__init__()
        self.l1 = self.two_conv_pool(1, 64, 64)
        self.l2 = self.two_conv_pool(64, 128, 128)
        self.l3 = self.three_conv_pool(128, 256, 256, 256)
        self.l4 = self.three_conv_pool(256, 256, 256, 256)

    def two_conv_pool(self, in_channels, f1, f2):
        s = nn.Sequential(
            nn.Conv2d(in_channels, f1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f1),
            nn.ReLU(inplace=True),
            nn.Conv2d(f1, f2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        for m in s.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        return s
    
    def three_conv_pool(self,in_channels, f1, f2, f3):
        s = nn.Sequential(
            nn.Conv2d(in_channels, f1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f1),
            nn.ReLU(inplace=True),
            nn.Conv2d(f1, f2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f2),
            nn.ReLU(inplace=True),
            nn.Conv2d(f2, f3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        for m in s.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        return s

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        
        return x

class SpinalVGG5(nn.Module):
    def __init__(self, num_classes = 10):
        super().__init__()
        self.vgg = VGG(num_classes)

        feat_dim = 256
        half_dim = feat_dim //2
        self.half_dim = half_dim

        self.linear1 = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(half_dim, feat_dim), nn.BatchNorm1d(feat_dim), nn.GELU()
        )
        self.linear2 = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(half_dim + feat_dim, feat_dim), nn.BatchNorm1d(feat_dim), nn.GELU()
        )
        self.linear3 = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(half_dim + feat_dim, feat_dim), nn.BatchNorm1d(feat_dim), nn.GELU()
        )
        self.linear4 = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(half_dim + feat_dim, feat_dim), nn.BatchNorm1d(feat_dim), nn.GELU()
        )
        self.proj = nn.Linear(4 * feat_dim, num_classes)


    def forward(self, x):
        x = self.vgg(x)
        x = x.view(x.size(0), -1)

        x1 = self.linear1(x[:, :self.half_dim])
        x2 = self.linear2(torch.cat([x[:, self.half_dim:], x1], dim = 1))
        x3 = self.linear3(torch.cat([x[:, :self.half_dim], x2], dim = 1))
        x4 = self.linear4(torch.cat([x[:, self.half_dim:], x3], dim = 1))

        x = torch.cat([x1, x2], dim = 1)
        x = torch.cat([x, x3], dim = 1)
        x = torch.cat([x, x4], dim =1)

        return self.proj(x)


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SpinalVGG5(10).to(DEVICE)


# Load Dataset and DataLoader

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
mnist_transform = [transforms.ToTensor(), transforms.Normalize(0.5, 0.5)]

mnist = datasets.MNIST(root = './data', download = True, transform = transforms.Compose(mnist_transform), train = True)
# print(len(mnist))
# dataloader = DataLoader(dataset= mnist, batch_size= 128, shuffle = True)

dataset, valid_dataset = torch.utils.data.random_split(mnist, [59000, 1000])

dataloader = DataLoader(dataset, batch_size =128, shuffle = True)
val_dataloader = DataLoader(valid_dataset, batch_size =128, shuffle = True)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Train

In [3]:
from tqdm.notebook import tqdm
import numpy as np
criterion = nn.CrossEntropyLoss()

optim = torch.optim.Adam(params = model.parameters(), lr = 1e-4)

for epoch in range(80):
    print("EPOCH : {}".format(epoch + 1))
    losses = []
    model.train()
    for i, (img, label) in enumerate(tqdm(dataloader)):
        img = img.to(DEVICE)
        label = label.to(DEVICE)

        pred = model(img)

        loss = criterion(pred, label)

        losses.append(loss.item())

        optim.zero_grad()
        loss.backward()
        optim.step()

    model.eval()
    scores = 0
    all_data = 0
    with torch.no_grad():
        for i , (img, label) in enumerate(tqdm(val_dataloader)):
            img = img.to(DEVICE)
            label = label.to(DEVICE)

            pred = model(img)

            arg = F.softmax(pred,dim=1).argmax(dim=1)
            score = (arg == label).sum().item()
            scores += score
            all_data += len(img)

    print("Loss : %f \t Score : %f"% (np.mean(losses), scores/all_data))



EPOCH : 1


  0%|          | 0/461 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


  0%|          | 0/8 [00:00<?, ?it/s]

Loss : 0.303420 	 Score : 0.993000
EPOCH : 2


...

  0%|          | 0/8 [00:00<?, ?it/s]

Loss : 0.001434 	 Score : 0.994000
EPOCH : 80


  0%|          | 0/461 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Loss : 0.001026 	 Score : 0.999000
